# Auth Log reader
This will read server logs saved by the pickleAuth addin

To get test data either visit a page or use:

In [20]:
import os
from datetime import datetime

def extract_timestamp(filename: str) -> str:
    # Extract the base filename without the directory
    base_name = os.path.basename(filename)
    
    # Split the filename to extract the timestamp part (assumes it's after 'auth_data_')
    timestamp_str = base_name.split('_')[2].replace('.pkl', '')
    
    # Convert to float (to handle sub-second precision)
    timestamp = float(timestamp_str)
    
    # Convert the timestamp to a human-readable datetime format
    readable_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S') 
    
    return readable_time

In [22]:
from pathlib import Path
import pickle 

authlogs = list(Path("data/auth_logs/").glob("*.pkl"))
testlog = authlogs[0]#"data/auth_logs/auth_data_1728130272.249.pkl"
with open(testlog, 'rb') as f:
    data = pickle.load(f)

# Inspect the data
print(f"File: {testlog} Time Saved: {extract_timestamp(testlog)}")
print(data)

File: data/auth_logs/auth_data_1728390503.895.pkl Time Saved: 2024-10-08 13:28:23
{'auth_payload': 'None', 'requested_page': '/batch/info', 'slide_id': '1027f5d5717058dbb6e31d4092132d78', 'plugin': 'None', 'slide': "SlideInfo(id='1027f5d5717058dbb6e31d4092132d78', channels=[SlideChannel(id=0, name='Red', color=SlideColor(r=255, g=0, b=0, a=0)), SlideChannel(id=1, name='Green', color=SlideColor(r=0, g=255, b=0, a=0)), SlideChannel(id=2, name='Blue', color=SlideColor(r=0, g=0, b=255, a=0))], channel_depth=8, extent=SlideExtent(x=146185, y=88889, z=1), num_levels=4, pixel_size_nm=SlidePixelSizeNm(x=262.598, y=262.598, z=None), tile_extent=SlideExtent(x=256, y=256, z=1), levels=[SlideLevel(extent=SlideExtent(x=146185, y=88889, z=1), downsample_factor=1.0), SlideLevel(extent=SlideExtent(x=36546, y=22222, z=1), downsample_factor=4.000036181612842), SlideLevel(extent=SlideExtent(x=9136, y=5555, z=1), downsample_factor=16.00130263792579), SlideLevel(extent=SlideExtent(x=2284, y=1388, z=1), dow

# Validating Header
Now that we have the Auth payload, let's validate it.

## Extra Imports

In [15]:
import time
import requests
import jwt
from botocore.exceptions import BotoCoreError, ClientError
from jwt.exceptions import DecodeError, ExpiredSignatureError
from pprint import pprint
from tqdm.auto import tqdm

## Settings

In [4]:
from dotenv import load_dotenv
load_dotenv("wsi_service/api/v3/integrations/.env")
idp_url = os.environ["idp_url"]
client_id = os.environ["client_id"]
jwks_url = os.environ["jwks_url"]
cognito_user_pool_id = os.environ["cognito_user_pool_id"]  # Add this to settings
aws_region = os.environ["aws_region"] # Add this to settings



## Special Function
Need to get the token from its' string representation due to the way it was saved:

In [5]:
import ast
auth_payload_str  = data['auth_payload']
auth_payload = auth_payload_str.strip("'")

token = auth_payload

## Functions

In [11]:
def validate_cognito_token(token):
    headers = jwt.get_unverified_header(token)
    kid = headers["kid"]

   
    response = requests.get(jwks_url)
    keys = response.json()["keys"]

    # Find the key that matches the kid in the JWT header
    key = next(k for k in keys if k["kid"] == kid)

    # Use the key to validate the token (you can use PyJWT or any other library here)
    public_key = jwt.algorithms.RSAAlgorithm.from_jwk(key)
    decoded_token = jwt.decode(token, public_key, algorithms=["RS256"],options={"verify_exp": False})

    return decoded_token

## Processing

In [12]:


try:
    # Validate the token against AWS Cognito
    decoded_token = validate_cognito_token( token)
    pprint(decoded_token)
    if decoded_token.get("client_id") != client_id:
        raise PermissionError("Invalid client ID")

    # Optionally, check custom claims or other parts of the token
    

except (DecodeError, ExpiredSignatureError) as e:
    raise PermissionError(f"Invalid token: {str(e)}")
except (BotoCoreError, ClientError) as e:
    raise PermissionError(f"Error validating token with Cognito: {str(e)}")

{'auth_time': 1728372597,
 'client_id': 'rf3988tf6jojhbfi9av6spi3g',
 'exp': 1728376197,
 'iat': 1728372597,
 'iss': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_x554UqUn0',
 'jti': '83e406eb-0c21-49b4-b578-5c42b184e014',
 'origin_jti': 'e1329a4d-ba0e-4115-a390-4872850d3339',
 'scope': 'openid email',
 'sub': '94284468-8051-706c-a1fb-6bd9a5d94e07',
 'token_use': 'access',
 'username': '94284468-8051-706c-a1fb-6bd9a5d94e07',
 'version': 2}


# Check all saved tokens
Let's make sure it's being sent with every request.

In [19]:
fails = 0
for log in tqdm(authlogs):
    with open(log, 'rb') as f:
        data = pickle.load(f)
    auth_payload_str  = data['auth_payload']
    token = auth_payload_str.strip("'")
    try:
        decoded_token = validate_cognito_token( token)
        if decoded_token.get("client_id") != client_id:
            fails +=1
    except:
        print("exception")
        print(f"File: {log} Time Saved: {extract_timestamp(log)}")
        pprint(data)
        fails +=1
print(f"There were {fails} failed authorisation(s) out of {len(authlogs)} tokens")

  0%|          | 0/74 [00:00<?, ?it/s]

exception
File: data/auth_logs_aman/auth_data_1728372598.274.pkl Time Saved: 2024-10-08 08:29:58
{'auth_payload': 'None',
 'plugin': 'None',
 'slide': "SlideInfo(id='82f04d466c2d5dcbae674e817caed0d7', "
          "channels=[SlideChannel(id=0, name='Red', color=SlideColor(r=255, "
          "g=0, b=0, a=0)), SlideChannel(id=1, name='Green', "
          'color=SlideColor(r=0, g=255, b=0, a=0)), SlideChannel(id=2, '
          "name='Blue', color=SlideColor(r=0, g=0, b=255, a=0))], "
          'channel_depth=8, extent=SlideExtent(x=13440, y=11520, z=1), '
          'num_levels=5, pixel_size_nm=SlidePixelSizeNm(x=459.7067071208569, '
          'y=459.7278411180581, z=None), tile_extent=SlideExtent(x=256, y=256, '
          'z=1), levels=[SlideLevel(extent=SlideExtent(x=13440, y=11520, z=1), '
          'downsample_factor=1.0), SlideLevel(extent=SlideExtent(x=6720, '
          'y=5760, z=1), downsample_factor=2.0), '
          'SlideLevel(extent=SlideExtent(x=3360, y=2880, z=1), '
          